In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "devkartik/gl_llm",
    # max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Unsloth unsuccessfully patched LoraLayer.update_layer. Please file a bug report.
Luckily, your training run will still work in the meantime!


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
alpaca_prompt = """Bhagwad gita's knowledge is paired with an input that provides further context related to gita only. Write a response that appropriately answers and elaborate the request.

### Input:
{}

### Response:
{}"""

In [ ]:
# alpaca_prompt = """Bhagwad gita's knowledge is paired with an input that provides further context related to gita only. Write a response that appropriately answers and elaborate the request.

# ### Input:
# {}

# ### Response:
# {}"""

# def extract_response(full_text):
#     # Assuming the response is after "### Response:\n"
#     full_text = full_text.replace("<|end_of_text|>","")
#     response_start = full_text.find("### Response:\n")
#     if response_start != -1:
#         return full_text[response_start + len("### Response:\n"):].strip()
#     return full_text

# def generate_response(input_text, model, tokenizer):
#     prompt = alpaca_prompt.format(input_text, "")
#     inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#     outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = False)
#     output = tokenizer.batch_decode(outputs)
#     responses = set()
#     while len(responses) < 5:
#         response = generate_response(input_text, model, tokenizer)
#         if len(response.split()) >= 2:
#             responses.add(extract_response(output[0]))
#     return list(responses)

In [ ]:
def generate_response_with_stream(input_text, model, tokenizer):
    prompt = alpaca_prompt.format(input_text, "")
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
    output = tokenizer.batch_decode(outputs)
    return extract_response(output[0])

In [ ]:
def extract_response(full_text):
    full_text = full_text.replace("<|end_of_text|>","")
    response_start = full_text.find("### Response:\n")
    if response_start != -1:
        return full_text[response_start + len("### Response:\n"):].strip()
    return full_text

In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
# input_text = "What is purpose of life?"
# # response = generate_response_with_stream(input_text, model, tokenizer)
# responses = set()
# while len(responses) < 5:
#     response = generate_response_with_stream(input_text, model, tokenizer)
#     responses.add(response)

In [ ]:
# responses

In [ ]:
input_text = "What is the meaning of the name krsna?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Kṛṣṇa means all-attractive.


In [ ]:
input_text = "who is elon musk?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

He's an American. He's very rich man.


In [ ]:
input_text = "what is the crux of gita?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

The crux is that the Supreme Personality of Godhead is the cause of everything.


In [ ]:
input_text = "What about gandhi's non-violence policies?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Non-violence is a very good policy.


In [ ]:
input_text = "What gita teaches us?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

The Supreme Personality of Godhead, He is the original person, and He is the cause of all causes. He is the original cause of everything. Therefore He is the cause of all causes. He is the supreme cause.


In [ ]:
input_text = "What about kalyuga?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Kalyuga is very bad.


In [ ]:
input_text = "What can we do if we are in a state of sadness?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

You have to find out the cause.


In [ ]:
input_text = "Tell me something about life?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Life is eternal.


In [ ]:
input_text = "What is the maha mantra?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

HareKṛṣṇa,HareKṛṣṇa,Kṛṣṇa,Kṛṣṇa,HareHare.HareRāma,HareRāma,Rāma,Rāma,HareHare.


In [ ]:
input_text = "tell me about iskcon?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Iskcon is the organization for spreading the knowledge of God.


In [ ]:
input_text = "why iskcon is started?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Because I wanted to spread Kṛṣṇaconsciousness all over the world.


In [ ]:
input_text = "how can we gain Kṛṣṇaconsciousness?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Kṛṣṇaconsciousness means Kṛṣṇais everything.


In [ ]:
input_text = "what are the steps to gain Kṛṣṇaconsciousness?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Kṛṣṇaconsciousness is not to be gained. Kṛṣṇaconsciousness is there. You have to revive it. It is already there.


In [ ]:
input_text = "what are the steps to gain Kṛṣṇaconsciousness?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

This is the first step, that you must learn how to chant. This is the first step.


In [ ]:
input_text = "what are the steps to gain Kṛṣṇaconsciousness?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

First of all, one must be faithful to the spiritual master. That is the first step. And then one must be faithful to the scriptures. That is the second step.
